### Import the packages needed (only random for our model)
### Specify where the training levels are
### Specify the name pattern for the training levels in that folder
### Specify the number of training levels to be used
### Specify the special tile used as the level boundary

In [1]:
from __future__ import print_function
import random
inputFolder = 'trainingData/'
levelNameBase = 'mario_'
numberOfTrainingLevels = 10
BOUNDARY_TILE = ':'

### Read in the training levels into 2-d lists, and store them

In [2]:
trainingLevels = list()
for levelIndex in range(1, numberOfTrainingLevels+1):
    try:
        with open(inputFolder + levelNameBase
                + str(levelIndex) + '.txt') as trainingLevel:

            level = trainingLevel.readlines()
            level = [ row.strip('\n') for row in level]

            trainingLevels.append(level)
    except IOError as err:
        print(err)

### Determine the set of tile types from the training levels

In [3]:
tileTypes = [BOUNDARY_TILE]
for level in trainingLevels:
    for row in level:
        for tile in row:
            if tile not in tileTypes:
                tileTypes.append(tile)

numberOfTileTypes = len(tileTypes)
print('Number of tile types:', numberOfTileTypes)

Number of tile types: 14


At this point, we've read in the training levels and determined the set of tile types that make up the domain. We cna now begin defining our model.

### First, we need to define the network structure of the MdMC

The network structure defines which surrounding tiles the value of the current position depends on. We define our network structures using a 2-d list. 

For example, a network structure of: 
    [[0,1,2],
     [0,0,1]] 
    
    states that the current position (2), relies on the tile to the left and the tile below (1's), and the 0's are used to ensure each sublist is of the same length.
    
Note, the more complex the network structure, the more data is needed to estimate the model. 

Furthermore, the "network complexity" is the number of states that the current state depends on (i.e., the number of 1's in the network structure).

We also record the position of the dependent state in the network structure to make training more concise later.

In [4]:
DEPENDENT_STATE = 2
INDEPENDENT_STATE = 1

networkStructure = [[0,1,2],
                    [0,1,1]]

networkComplexity = 0

for y in range(len(networkStructure)):
    for x in range(len(networkStructure[y])):
        if networkStructure[y][x] == 1:
            networkComplexity += 1
        if networkStructure[y][x] == 2:
            dependentPosition_X = x
            dependentPosition_Y = y
print('Network Structure: ', networkStructure)
print('Network Complexity: ', networkComplexity)
print('Position of dependent state: ', dependentPosition_Y, dependentPosition_X)

Network Structure:  [[0, 1, 2], [0, 1, 1]]
Network Complexity:  3
Position of dependent state:  0 2


### We can now begin training our model

We first define the needed size of the conditional probability distribution (CPD), which is dependent on the network complexity and number of tile types. The dimensions of the CPD are:

    (numberOfTileTypes^networkComplexity) X (numberOfTileTypes +1)
    
The first dimension is the number of possible surrounding tile configurations, and the second dimension is the number of tile types plus an additional space to track the total number of times each configuration occurs.

We then train our model by iterating through all the training levels and counting how many times each tile type follows each configuration  and how many times each configuration occurs.

Finally, we estimate the probabilities by dividing the number of times each tile follows each configuration by the number of times that configuration occurred.

Note, we define two helper functions to determine the surrounding tile configuration and its index into the CPD.

In [5]:
def GetSurroundingTiles(networkStructure, level, xPos, yPos, dependentPosition_X, 
                        dependentPosition_Y):
    surroundingTiles = list()
    for y in range(len(networkStructure)):
        for x in range(len(networkStructure[0])):
            if networkStructure[y][x] == INDEPENDENT_STATE:
                if ((yPos + (y - dependentPosition_Y)) < len(level) 
                    and (xPos - (dependentPosition_X - x)) >= 0):
                
                    surroundingTiles.append(level[yPos + (y - dependentPosition_Y)]
                                        [xPos - (dependentPosition_X - x)])
                else:
                    surroundingTiles.append(BOUNDARY_TILE)

    return surroundingTiles

def ConvertConfigurationToIndex(surroundingTiles, tileTypes):

    exponent = 0
    configurationIndex = 0
    for t in surroundingTiles:
        configurationIndex += tileTypes.index(t)*pow(len(tileTypes), exponent)
        exponent += 1

    return configurationIndex

tableHeight = pow(numberOfTileTypes,networkComplexity)

# The width of the table is the number of tile types + 1 to account for
# each tile type and the totals needed to estimate the probability.
tableWidth = numberOfTileTypes+1

CPD = [[0.0 for x in range(tableWidth)] for y in range(tableHeight)]

# We now iterate through each training level and record how many times 
# each tile follows each tile configuration, and the total number of times
# each configuration appears.
for level in trainingLevels:
    # We start at the bottom of the level and move upward.
    for y in range(len(level)-1, -1, -1):
        for x in range(len(level[y])):
            surroundingTiles = GetSurroundingTiles(networkStructure, level, x, y, 
                                                   dependentPosition_X, dependentPosition_Y)

            configurationIndex = ConvertConfigurationToIndex(surroundingTiles, tileTypes)

            CPD[configurationIndex][tileTypes.index(level[y][x])] += 1
            CPD[configurationIndex][len(tileTypes)] += 1


# Once the totals have been recorded, we estimate the probabilities
# Notice, if a configuration has not been observed, we set a uniform
# probability for all tile types to follow for that configuration, except
# the special border tile which should never be sampled.
for row in range(len(CPD)):
    for position in range(len(CPD[row])-1):
        if CPD[row][len(tileTypes)] > 0:
            CPD[row][position] = CPD[row][position]/CPD[row][len(tileTypes)]
        elif position > 0:
            CPD[row][position] = 1.0/(len(tileTypes)-1)
            
print(len(CPD), len(CPD[0]))

2744 15


### After training, all thats left to do is sample new levels

To sample a new level we need to define the dimensions of the output level, as well as the length of the look ahead.

Then we sample a new level, tile by tile starting in the bottom left and completing row by row.

We choose a tile probabilistically by determining the surrounding tile configuration and sampling from the trained distribution. We then perform a recursive look ahead in order to check future positions for unseen tile congifurations. If such configurations are encountered, we resample the tile.

We define a function for recursively sampling the correct number of tiles ahead, and a function for sampling a full level.

Additionally, in order to sample a new level, we need to define the length of the look ahead to be used.

In [6]:
height = 14
length = 110
lookAhead =3
def SampleTile(CPD, networkStructure, dependentPosition_X, dependentPosition_Y, level, 
               xPosInLevel, yPosInLevel, possibleTileTypes, allTileTypes, lookAhead, 
               numberGenerator):

    # If we reach the end of the look ahead, or the end of a row
    # then the look ahead succeeded.
    if lookAhead == -1  or xPosInLevel >= len(level[yPosInLevel]):
        return True


    surroundingTiles = GetSurroundingTiles(networkStructure, level, xPosInLevel, yPosInLevel, 
                                           dependentPosition_X, dependentPosition_Y)

    configurationIndex = ConvertConfigurationToIndex(surroundingTiles, allTileTypes)

    # Make a copy of the list of possible tiles and the probability distribution
    # for that configuration, so that they can be modified without modifying
    # the original.
    remainingTileTypes = list(possibleTileTypes)
    distribution = list(CPD[configurationIndex])


    # If at any depth into the lookahead, we reach an unseen configuration
    # of tile types then we need to backtrack.
    if distribution[len(distribution)-1] == 0:
        return False


    # Otherwise, we sample a tile and continue with the look ahead.
    while len(remainingTileTypes) > 1:
        # First sample a tile for the current position.
        choice = numberGenerator.uniform(0,1)
        prob = 0
        for index in range(1,len(remainingTileTypes)):
            prob += distribution[index]
            if choice <= prob:
                level[yPosInLevel][xPosInLevel] = remainingTileTypes[index]
                break

        # If the position was sampled successfully, then the look ahead
        # must have succeeded at each subsequent position.
        if SampleTile(CPD, networkStructure, dependentPosition_X, dependentPosition_Y, level, 
                      xPosInLevel+1, yPosInLevel, allTileTypes, allTileTypes, lookAhead-1, 
                      numberGenerator):
            return True

        # If the position was not sampled successfully, the we remove the 
        # chosen tile from the list of possible tiles, normalize the probability 
        #distribution, and resample the current and subsequent positions.
        else:
            tileIndex = remainingTileTypes.index(level[yPosInLevel][xPosInLevel])
            del remainingTileTypes[tileIndex]
            del distribution[tileIndex]
            
            # Re-estimate the distribution
            sum=0
            for index in range(0, len(distribution)-1):
                sum += distribution[index]

            # If none of the remaining tile types have been observed following
            # this configuration, then this is essentially an unseen 
            # configuration, so we backtrack.
            if sum == 0:
                return False
            for index in range(0, len(distribution)-1):
                distribution[index] = distribution[index]/sum


    # If we run out of tiles to choose from, then backtrack.
    return False



def SampleLevel(length, height, numberGenerator, CPD, networkStructure, dependentPosition_X, 
                dependentPosition_Y, tileTypes, lookAhead):
    
    outputLevel = [[' ' for x in range(length)] for y in range(height)]

    for y in range(height-1, -1, -1):
        for x in range(length):
            successfullySampled = SampleTile(CPD, networkStructure, dependentPosition_X, 
                                             dependentPosition_Y, outputLevel,x, y, tileTypes, 
                                             tileTypes, lookAhead, numberGenerator)

            # If the look ahead fails at the top level, choose a tile at random
            if not successfullySampled:
                choice = numberGenerator.randint(1,len(tileTypes)-1)
                outputLevel[y][x] = tileTypes[choice]

    return outputLevel

rand = random.SystemRandom()

outputLevel = SampleLevel(length, height, rand, CPD, networkStructure, dependentPosition_X, dependentPosition_Y,
           tileTypes, lookAhead)

for y in range(len(outputLevel)):
    for x in range(len(outputLevel[y])):
        print(outputLevel[y][x], end='')
    print()



------------------------------------------------------SS-----------------------------E------------------------
-X--------------------------------------------------------------------XX-------------XXXXX--------------------
--------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------o-----------------------------------
---------------------------------------------------------------------?SS--------------------------------------
--------------------------------------------------------------------------------------------------------------
-----------------------------------------?----SSSSSSSSSSS-----------------------------------------------------
--------------------E-----SSQ-----------------SSS-------------------------------------------------------------
---------------------------------------------------QQQ--------------------------------------------------------
-

Notice, occasionally sections of seamingly nonsensical tile will appear in the output. This is because the look ahead has failed at a nearby position, and a tile was sampled at random. This random tile then led to many other look ahead failures and therefore more randomly sampled tiles.

In our work we typically sample the tile using a simpler model (i.e. trained with a simpler network structure) instead of randomly sampling the tile in order to avoid these situations. We have left that out of this tutorials code for clarity.